In [56]:
import pandas as pd
import json
import numpy as np

In [57]:
# Inport the CSV files from disk
movies_metadata_path = '../../Boot_Camp_files/Module_8/movies_metadata.csv'
movies_metadata_df = pd.read_csv(movies_metadata_path, low_memory=False)
ratings_path = '../../Boot_Camp_files/Module_8/ratings.csv'
ratings_df = pd.read_csv(ratings_path)
ratings_df.head()


,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [93]:
with open('wikipedia-movies (1).json',mode = 'r') as file:
    wiki_movies_raw = json.load(file)

In [94]:
len(wiki_movies_raw)

7311

In [95]:
# Convert the list of dictionaries to a df to inspect
wiki_movies_raw_df = pd.DataFrame(wiki_movies_raw)

In [96]:
wiki_movies_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7311 entries, 0 to 7310
Columns: 193 entries, url to Polish
dtypes: float64(1), object(192)
memory usage: 10.8+ MB


In [97]:
print(sorted(wiki_movies_raw_df.columns.tolist()))

['Actor control', 'Adaptation by', 'Alias', 'Alma mater', 'Also known as', 'Animation by', 'Arabic', 'Area', 'Area served', 'Artist(s)', 'Attraction type', 'Audio format', 'Author', 'Based on', 'Biographical data', 'Bopomofo', 'Born', 'Box office', 'Budget', 'Camera setup', 'Cantonese', 'Characters', 'Children', 'Chinese', 'Cinematography', 'Closing date', 'Color process', 'Comics', 'Composer(s)', 'Coordinates', 'Country', 'Country of origin', 'Cover artist', 'Created by', 'Date premiered', 'Designer(s)', 'Developed by', 'Developer(s)', 'Dewey Decimal', 'Died', 'Directed by', 'Director', 'Distributed by', 'Distributor', 'Divisions', 'Duration', 'Edited by', 'Editor(s)', 'Ending theme', 'Engine', 'Engine(s)', 'Executive producer(s)', 'Family', 'Fate', 'Film(s)', 'Followed by', 'Format(s)', 'Formerly', 'Founded', 'Founder', 'Founders', 'French', 'Full name', 'Gender', 'Genre', 'Genre(s)', 'Genres', 'Gwoyeu Romatzyh', 'Hangul', 'Hanyu Pinyin', 'Headquarters', 'Hebrew', 'Height', 'Hepburn'

Use list comprehension to cut the raw movies read to only those that have a director (either 'Director' or 'Directed by'), have a url for an imdb entry, and do not have any values under "No. of episodes"

In [98]:

wiki_movies = [movie for movie in wiki_movies_raw 
               if ('Director' in movie or 'Directed by' in movie) and 
               'imdb_link' in movie and
              'No. of episodes' not in movie]

In [99]:
len(wiki_movies)

7076

I identified column names in the wiki_movies list of dictionaries that are alternate names for movies.  THe function is passed a dictionary for a single movie.  If one of the identified column names has data for a dictionary then that value is saved to a separate dictionary and the value is removed from the movie database.  If there was an alternate title then a new key value pair is added to the movie.

Going back to add a sub function to the clean_movie function.  For columns that have the same content but slightly different names the column names are changed to get rid of the excess columns.

I use a dictionary of the columns to be removed and the column name for the change.

In [100]:
def clean_movie(movie):
    local_movie = dict(movie)
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in local_movie:
            alt_titles[key] = local_movie[key]
            local_movie.pop(key)
    if len(alt_titles) > 0:
        local_movie['alt_titles'] = alt_titles

#define embedded function to remove a column that is equivalent to another column and pop the value to the remaining column
    def change_column_name (old_key,new_key):
        if old_key in local_movie:
            local_movie[new_key] = local_movie.pop(old_key)
# Dictionary of repeated (nearly) column names and what they should be changed to
    old_key_dict = {'Directed by':'Director',
                   'Country of origin':'Country',
                   'Distributed by':'Distributor',
                   'Edited by':'Editor(s)',
                   'Produced by':'Producer',
                    'Producer(s)':'Producer',
                    'Production company(s)':'Productioncompany',
                    'Productioncompanies ':'Productioncompany',
                    'Released':'Release date',
                    'Running time':'Length',
                    'Screen story by':'Writer(s)',
                    'Screenplay by':'Writer(s)',
                   'Release Date':'Release date,',
                   'Story by':'Writer(s)',
                   'Theme music composer':'Composer(s)',
                   'Written by':'Writer(s)'}
    
    for key,value in old_key_dict.items():
       change_column_name(key,value)
                    
    return local_movie

In [101]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]

In [102]:
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Created by',
 'Director',
 'Distributor',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Length',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Picture format',
 'Preceded by',
 'Producer',
 'Production location(s)',
 'Productioncompany',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Starring',
 'Suggested by',
 'Venue',
 'Voices of',
 'Writer(s)',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']

In [103]:
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Narrated by,Music by,Cinematography,Productioncompany,...,Created by,Preceded by,Adaptation by,Suggested by,alt_titles,Recorded,Venue,Label,Animation by,Color process
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...","Andrew ""Dice"" Clay","[Cliff Eidelman, Yello]",Oliver Wood,Silver Pictures,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",NaN,Maurice Jarre,Mark Plummer,Avenue Pictures,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",NaN,Charles Gross,Roger Deakins,"[Carolco Pictures, IndieProd Company]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",NaN,NaN,Carlo Di Palma,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",NaN,Maurice Jarre,Russell Boyd,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
